# IMDB project: ETL for EDA
### by Javier Alonso

In this Jupyter Notebook is created the ETL and part of the EDA of "My IMDB project".

The database was downloaded from IMDB on 2020/05/25 but today, three days after, they have changed all the system for retrieving the data, so I won´t be able to upgrade easily the project with new data.

The content of the EDA is published in my [blog](https://javieralonsoalonso.com/imdb-project-exploratory-data-analysis-eda/)

## Table of Contents

1. [Preliminary Wrangling](#Preliminary_Wrangling)<br>
1.a.-[Gathering Data](#Gathering_Data)<br>
1.b.-[Assesing and Cleaning Data](#Assesing_cleaning_data)<br>
1.c.-[Consolidation of Dataframes](#Consolidaion_dataframes)<br>
2. [Exploratory Data Analysis](#EDA)<br>

## <a class="anchor" id="Preliminary_Wrangling">Part 1: Preliminary Wrangling

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import f1_score

from sklearn.preprocessing import MinMaxScaler

### <a class="anchor" id="Gathering_Data"> 1.a.- Gathering Data

In this section we´ll gather data from both files available

In [193]:
#Basic information on the title
df_basics = pd.read_csv("title.basics.tsv", low_memory=False, sep="\t", na_values=["\\N","nan"])

#Information of the people in movies (director, actor,...)
df_names = pd.read_csv("name.basics.tsv", sep="\t", na_values=["\\N","nan"])

#Contains the principal cast/crew for titles
df_principals = pd.read_csv("title.principals.tsv",  sep="\t", na_values=["\\N","nan"])

#director and writer information
df_crew = pd.read_csv("title.crew.tsv",  sep="\t", na_values=["\\N","nan"])

# Contains the IMDb rating and votes information for titles
df_ratings = pd.read_csv("title.ratings.tsv", sep="\t", na_values=["\\N","nan"])

#Information of the title of the movies in different areas and zones. We won´t use it
# df_movie_names = pd.read_csv("title.akas.tsv", low_memory=False, sep="\t", na_values=["\\N","nan"])

#Contains the tv episode information. We won´t use it
# df_episodes = pd.read_csv("title.episode.tsv",  sep="\t", na_values=["\\N","nan"])

### <a class="anchor" id="Assesing_cleaning_data"> 1.b.- Asessing and cleaning data

Now, we´re going to assess the dataframe visually and programmatically for quality and tidiness issues, and do the cleaning of these issues

#### MOVIES DATABASE 

In [194]:
info = df_basics.copy()

In [195]:
info.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"Comedy,Short"


In [196]:
info.shape

(6831547, 9)

#### Filtering info

- keeping only the movies

In [197]:
info.titleType.value_counts()

tvEpisode       4869408
short            741081
movie            551301
video            265727
tvSeries         184466
tvMovie          121175
tvMiniSeries      31078
tvSpecial         29209
videoGame         25548
tvShort           12554
Name: titleType, dtype: int64

In [198]:
movies = info[info.titleType=='movie']
movies = movies.drop(['titleType'], axis=1)

- deleting adult movies

In [199]:
movies.isAdult.value_counts()

0    542500
1      8801
Name: isAdult, dtype: int64

In [200]:
movies = movies[movies.isAdult==0]
movies = movies.drop(['isAdult'], axis=1)

- keeping just the original title

In [201]:
movies = movies.drop(['primaryTitle'], axis=1)

- deleting the endYear (only used for TV Shows)

In [202]:
movies.endYear.value_counts()

Series([], Name: endYear, dtype: int64)

In [203]:
movies = movies.drop(['endYear'], axis=1)

- change column names+

In [204]:
movies.columns = ['idMovie', 'originalTitle', 'Year', 'Minutes', 'Genres']

In [205]:
movies.head()

,idMovie,originalTitle,Year,Minutes,Genres
8,tt0000009,Miss Jerry,1894.0,45,Romance
145,tt0000147,The Corbett-Fitzsimmons Fight,1897.0,20,"Documentary,News,Sport"
332,tt0000335,Soldiers of the Cross,1900.0,NaN,"Biography,Drama"
499,tt0000502,Bohemios,1905.0,100,NaN
571,tt0000574,The Story of the Kelly Gang,1906.0,70,"Biography,Crime,Drama"


In [206]:
movies.shape

(542500, 5)

- add the ratings

In [207]:
ratings = df_ratings.copy()

In [208]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1615
1,tt0000002,6.0,197
2,tt0000003,6.5,1296
3,tt0000004,6.1,121
4,tt0000005,6.1,2075


In [209]:
movies.shape

(542500, 5)

In [210]:
movies.head()

,idMovie,originalTitle,Year,Minutes,Genres
8,tt0000009,Miss Jerry,1894.0,45,Romance
145,tt0000147,The Corbett-Fitzsimmons Fight,1897.0,20,"Documentary,News,Sport"
332,tt0000335,Soldiers of the Cross,1900.0,NaN,"Biography,Drama"
499,tt0000502,Bohemios,1905.0,100,NaN
571,tt0000574,The Story of the Kelly Gang,1906.0,70,"Biography,Crime,Drama"


In [211]:
movies = pd.merge(movies, ratings, left_on = 'idMovie', right_on = 'tconst', how = 'left')
movies = movies.drop(['tconst'], axis=1)

In [212]:
movies.shape

(542500, 7)

In [213]:
movies.head()

,idMovie,originalTitle,Year,Minutes,Genres,averageRating,numVotes
0,tt0000009,Miss Jerry,1894.0,45,Romance,5.9,153.0
1,tt0000147,The Corbett-Fitzsimmons Fight,1897.0,20,"Documentary,News,Sport",5.2,346.0
2,tt0000335,Soldiers of the Cross,1900.0,NaN,"Biography,Drama",6.1,40.0
3,tt0000502,Bohemios,1905.0,100,NaN,3.8,6.0
4,tt0000574,The Story of the Kelly Gang,1906.0,70,"Biography,Crime,Drama",6.1,574.0


#### Keeping only movies with votes

In [214]:
movies.shape

(542500, 7)

In [215]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 542500 entries, 0 to 542499
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   idMovie        542500 non-null  object 
 1   originalTitle  542500 non-null  object 
 2   Year           478643 non-null  float64
 3   Minutes        343594 non-null  object 
 4   Genres         472736 non-null  object 
 5   averageRating  244841 non-null  float64
 6   numVotes       244841 non-null  float64
dtypes: float64(3), object(4)
memory usage: 33.1+ MB


In [216]:
movies = movies[movies['numVotes'].notna()]

In [217]:
movies.shape

(244841, 7)

#### Fixing Null Values

In [218]:
movies.isnull().sum()/len(movies)

idMovie          0.000000
originalTitle    0.000000
Year             0.000045
Minutes          0.108442
Genres           0.040606
averageRating    0.000000
numVotes         0.000000
dtype: float64

Year will be replaced with the median and minutes (length) with the mean

In [219]:
movies['Minutes'].fillna((movies['Minutes'].median()), inplace=True)
movies['Year'].fillna((movies['Year'].mean()), inplace=True)
movies['Genres'].fillna('No available', inplace=True)

In [220]:
movies.isnull().sum()/len(movies)

idMovie          0.0
originalTitle    0.0
Year             0.0
Minutes          0.0
Genres           0.0
averageRating    0.0
numVotes         0.0
dtype: float64

#### Data types

In [221]:
movies.Year = movies.Year.astype('int64')
movies.Minutes = movies.Minutes.astype('int64')
movies.numVotes = movies.numVotes.astype('int64')

In [222]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244841 entries, 0 to 542493
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   idMovie        244841 non-null  object 
 1   originalTitle  244841 non-null  object 
 2   Year           244841 non-null  int64  
 3   Minutes        244841 non-null  int64  
 4   Genres         244841 non-null  object 
 5   averageRating  244841 non-null  float64
 6   numVotes       244841 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 14.9+ MB


#### Feature engineering and dummy variables

Out of the year, we obtain the **decade**. And we get the dummies for genre, country and languaje because these fields have many values in it separated by commas

In [223]:
movies['decade'] = movies.Year//10*10

In [224]:
movies.head()

,idMovie,originalTitle,Year,Minutes,Genres,averageRating,numVotes,decade
0,tt0000009,Miss Jerry,1894,45,Romance,5.9,153,1890
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,346,1890
2,tt0000335,Soldiers of the Cross,1900,91,"Biography,Drama",6.1,40,1900
3,tt0000502,Bohemios,1905,100,No available,3.8,6,1900
4,tt0000574,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",6.1,574,1900


In [225]:
len(movies)

244841

#### MOVIES CREW

In [226]:
df_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography,NaN
3,tt0000002,1,nm0721526,director,NaN,NaN
4,tt0000002,2,nm1335271,composer,NaN,NaN


In [227]:
df_principals.category.value_counts()

actor                  9105285
actress                6816303
self                   6638020
writer                 5244946
director               4552840
producer               2396946
composer               1414656
cinematographer        1395674
editor                 1312898
production_designer     297712
archive_footage         231023
archive_sound             2383
Name: category, dtype: int64

In [228]:
df_names.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous","tt0050419,tt0072308,tt0053137,tt0043044"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack","tt0071877,tt0037382,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,producer","tt0054452,tt0059956,tt0057345,tt0049189"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,soundtrack,writer","tt0080455,tt0078723,tt0072562,tt0077975"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0083922,tt0060827,tt0050986,tt0050976"


In [229]:
principals = pd.merge(df_principals, df_names[['nconst','primaryName']], left_on = 'nconst', right_on = 'nconst', how = 'left')

In [230]:
principals = principals[['tconst','primaryName','category']]

In [231]:
principals.head()

,tconst,primaryName,category
0,tt0000001,Carmencita,self
1,tt0000001,William K.L. Dickson,director
2,tt0000001,William Heise,cinematographer
3,tt0000002,Émile Reynaud,director
4,tt0000002,Gaston Paulin,composer


In [47]:
principals.category.value_counts()

actor                  9105285
actress                6816303
self                   6638020
writer                 5244946
director               4552840
producer               2396946
composer               1414656
cinematographer        1395674
editor                 1312898
production_designer     297712
archive_footage         231023
archive_sound             2383
Name: category, dtype: int64

In [48]:
directors = principals[principals.category == 'director']
actors = principals[principals.category == 'actor']
actress = principals[principals.category == 'actress']
writers = principals[principals.category == 'writer']

#### Directors info

In [232]:
directors = principals[principals.category == 'director']

directors.primaryName = '\'' + directors.primaryName.astype(str) + '\''
directors = directors[['tconst','primaryName']].groupby('tconst')['primaryName'].apply(','.join).reset_index()
directors['primaryName'] = directors['primaryName'].str.replace('\'', '')

C:\Users\cance\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [233]:
movies = pd.merge(movies, directors , left_on = 'idMovie', right_on = 'tconst', how = 'left')
movies = movies.drop(['tconst'], axis=1)
movies=movies.rename(columns = {'primaryName':'director'})

In [236]:
len(movies)

244841

In [234]:
movies.head()

,idMovie,originalTitle,Year,Minutes,Genres,averageRating,numVotes,decade,director
0,tt0000009,Miss Jerry,1894,45,Romance,5.9,153,1890,Alexander Black
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,346,1890,Enoch J. Rector
2,tt0000335,Soldiers of the Cross,1900,91,"Biography,Drama",6.1,40,1900,"Herbert Booth,Joseph Perry"
3,tt0000502,Bohemios,1905,100,No available,3.8,6,1900,Ricardo de Baños
4,tt0000574,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",6.1,574,1900,Charles Tait


#### Actors info

In [237]:
actors = principals[principals.category == 'actor']

actors.primaryName = '\'' + actors.primaryName.astype(str) + '\''
actors = actors[['tconst','primaryName']].groupby('tconst')['primaryName'].apply(','.join).reset_index()
actors['primaryName'] = actors['primaryName'].str.replace('\'', '')

C:\Users\cance\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [238]:
movies = pd.merge(movies, actors , left_on = 'idMovie', right_on = 'tconst', how = 'left')
movies = movies.drop(['tconst'], axis=1)
movies=movies.rename(columns = {'primaryName':'actors'})

In [239]:
len(movies)

244841

In [240]:
movies.head()

,idMovie,originalTitle,Year,Minutes,Genres,averageRating,numVotes,decade,director,actors
0,tt0000009,Miss Jerry,1894,45,Romance,5.9,153,1890,Alexander Black,"William Courtenay,Chauncey Depew"
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,346,1890,Enoch J. Rector,NaN
2,tt0000335,Soldiers of the Cross,1900,91,"Biography,Drama",6.1,40,1900,"Herbert Booth,Joseph Perry","Harold Graham,Mr. Graham,John Jones,Orrie Perr..."
3,tt0000502,Bohemios,1905,100,No available,3.8,6,1900,Ricardo de Baños,"Antonio del Pozo,El Mochuelo"
4,tt0000574,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",6.1,574,1900,Charles Tait,"John Tait,Norman Campbell"


#### Actress info

In [241]:
actress = principals[principals.category == 'actress']

actress.primaryName = '\'' + actress.primaryName.astype(str) + '\''
actress = actress[['tconst','primaryName']].groupby('tconst')['primaryName'].apply(','.join).reset_index()
actress['primaryName'] = actress['primaryName'].str.replace('\'', '')

C:\Users\cance\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [242]:
movies = pd.merge(movies, actress , left_on = 'idMovie', right_on = 'tconst', how = 'left')
movies = movies.drop(['tconst'], axis=1)
movies=movies.rename(columns = {'primaryName':'actress'})

In [243]:
len(movies)

244841

In [244]:
movies.head()

,idMovie,originalTitle,Year,Minutes,Genres,averageRating,numVotes,decade,director,actors,actress
0,tt0000009,Miss Jerry,1894,45,Romance,5.9,153,1890,Alexander Black,"William Courtenay,Chauncey Depew",Blanche Bayliss
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,346,1890,Enoch J. Rector,NaN,NaN
2,tt0000335,Soldiers of the Cross,1900,91,"Biography,Drama",6.1,40,1900,"Herbert Booth,Joseph Perry","Harold Graham,Mr. Graham,John Jones,Orrie Perr...",Beatrice Day
3,tt0000502,Bohemios,1905,100,No available,3.8,6,1900,Ricardo de Baños,"Antonio del Pozo,El Mochuelo",NaN
4,tt0000574,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",6.1,574,1900,Charles Tait,"John Tait,Norman Campbell","Elizabeth Tait,Bella Cola"


#### Writers info

In [245]:
writers = principals[principals.category == 'writer']

writers.primaryName = '\'' + writers.primaryName.astype(str) + '\''
writers = writers[['tconst','primaryName']].groupby('tconst')['primaryName'].apply(','.join).reset_index()
writers['primaryName'] = actress['primaryName'].str.replace('\'', '')

C:\Users\cance\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [246]:
movies = pd.merge(movies, writers , left_on = 'idMovie', right_on = 'tconst', how = 'left')
movies = movies.drop(['tconst'], axis=1)
movies=movies.rename(columns = {'primaryName':'writers'})

In [258]:
movies['director'].fillna('Not available', inplace=True)
movies['actors'].fillna('Not available', inplace=True)
movies['actress'].fillna('Not available', inplace=True)
movies['writers'].fillna('Not available', inplace=True)

In [259]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244841 entries, 0 to 244840
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   idMovie        244841 non-null  object 
 1   originalTitle  244841 non-null  object 
 2   Year           244841 non-null  int64  
 3   Minutes        244841 non-null  int64  
 4   Genres         244841 non-null  object 
 5   averageRating  244841 non-null  float64
 6   numVotes       244841 non-null  int64  
 7   decade         244841 non-null  int64  
 8   director       244841 non-null  object 
 9   actors         244841 non-null  object 
 10  actress        244841 non-null  object 
 11  writers        244841 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 24.3+ MB


#### Deleting documentaries

In [13]:
movies = movies[movies.Genres != 'Documentary']

In [14]:
movies.Genres.value_counts()

Drama                          42485
Comedy                         22287
No available                    9942
Comedy,Drama                    8044
Drama,Romance                   7559
                               ...  
History,Mystery                    1
Documentary,Fantasy,Mystery        1
Comedy,Film-Noir,Mystery           1
Crime,Film-Noir                    1
Adventure,Romance,Sport            1
Name: Genres, Length: 1172, dtype: int64

In [15]:
len(movies)

221748

In [16]:
movies.head()

,idMovie,originalTitle,Year,Minutes,Genres,averageRating,numVotes,decade,director,actors,actress,writers
0,tt0000009,Miss Jerry,1894,45,Romance,5.9,153,1890,Alexander Black,"William Courtenay,Chauncey Depew",Blanche Bayliss,Not available
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,346,1890,Enoch J. Rector,Not available,Not available,Not available
2,tt0000335,Soldiers of the Cross,1900,91,"Biography,Drama",6.1,40,1900,"Herbert Booth,Joseph Perry","Harold Graham,Mr. Graham,John Jones,Orrie Perr...",Beatrice Day,Not available
3,tt0000502,Bohemios,1905,100,No available,3.8,6,1900,Ricardo de Baños,"Antonio del Pozo,El Mochuelo",Not available,"Marguerite Thévenard,Bleuette Bernon"
4,tt0000574,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",6.1,574,1900,Charles Tait,"John Tait,Norman Campbell","Elizabeth Tait,Bella Cola",Not available


In [17]:
movies.shape

(221748, 12)

#### MOVIES GENRES

In [18]:
val = pd.concat([movies.idMovie, 
                 movies.Genres.str.split(',', expand=True)], 
                axis=1)
m = pd.melt(val, id_vars=['idMovie']).loc[:, ['idMovie', 'value']]
m.columns = ['idMovie', 'Genre']
m = m.dropna()

In [19]:
m.sort_values(by ='idMovie')

,idMovie,Genre
0,tt0000009,Romance
443497,tt0000147,Sport
221749,tt0000147,News
1,tt0000147,Documentary
221750,tt0000335,Drama
...,...,...
665241,tt9916132,History
443494,tt9916428,History
221746,tt9916428,Adventure
665242,tt9916428,War


In [20]:
genres = pd.merge(m, movies, left_on = 'idMovie', right_on = 'idMovie', how = 'left')
genres = genres.drop(['Genres'], axis=1)

- we delete the documentaries and adults

In [24]:
genres = genres[genres.Genre != 'Documentary']

In [25]:
genres.Genre.value_counts()

Drama           109905
Comedy           62150
Romance          28010
Action           26934
Crime            21014
Thriller         19905
Horror           17134
Adventure        16748
No available      9942
Family            9535
Mystery           9044
Biography         7429
Fantasy           7128
History           6752
Music             6006
Sci-Fi            5916
Musical           5662
War               5495
Animation         4241
Western           3885
Sport             2864
Film-Noir          785
News               666
Reality-TV          33
Adult                8
Talk-Show            3
Game-Show            1
Name: Genre, dtype: int64

In [27]:
genres.head()

,idMovie,Genre,originalTitle,Year,Minutes,averageRating,numVotes,decade,director,actors,actress,writers
0,tt0000009,Romance,Miss Jerry,1894,45,5.9,153,1890,Alexander Black,"William Courtenay,Chauncey Depew",Blanche Bayliss,Not available
2,tt0000335,Biography,Soldiers of the Cross,1900,91,6.1,40,1900,"Herbert Booth,Joseph Perry","Harold Graham,Mr. Graham,John Jones,Orrie Perr...",Beatrice Day,Not available
3,tt0000502,No available,Bohemios,1905,100,3.8,6,1900,Ricardo de Baños,"Antonio del Pozo,El Mochuelo",Not available,"Marguerite Thévenard,Bleuette Bernon"
4,tt0000574,Biography,The Story of the Kelly Gang,1906,70,6.1,574,1900,Charles Tait,"John Tait,Norman Campbell","Elizabeth Tait,Bella Cola",Not available
5,tt0000615,Drama,Robbery Under Arms,1907,91,4.5,15,1900,Charles MacMahon,"Jim Gerald,George Merriman,Lance Vane,William ...",Rhoda Dendron,Gene Gauntier


In [28]:
len(genres)

387195

### <a class="anchor" id="Consolidaion_dataframes"> 1.c.- Consolidation of dataframes

In this section we´re going to consolidate both dataframes into files, that we´ll read for the next sections

In [29]:
movies.to_csv('IMDB_clean_movies.csv', index = False)
genres.to_csv('IMDB_clean_genres.csv', index = False)

## <a class="anchor" id="EDA">Part 2: Exploratory Data Analysis (EDA)

In this section, we´ll investigate the distributions variables in the big dataframe. 

We won´t investigate the columns of passenger id, as it is just an identifier, and its value in a univariate exploration doesn´t make sense, and we won´t either investigate the Survived column as it´s incomplete

First we´ll import the data stored previously in files

In [30]:
movies = pd.read_csv('IMDB_clean_movies.csv')
genres = pd.read_csv('IMDB_clean_genres.csv')

In [39]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221748 entries, 0 to 221747
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   idMovie        221748 non-null  object 
 1   originalTitle  221748 non-null  object 
 2   Year           221748 non-null  int64  
 3   Minutes        221748 non-null  int64  
 4   Genres         221748 non-null  object 
 5   averageRating  221748 non-null  float64
 6   numVotes       221748 non-null  int64  
 7   decade         221748 non-null  int64  
 8   director       221748 non-null  object 
 9   actors         221746 non-null  object 
 10  actress        221740 non-null  object 
 11  writers        221743 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 20.3+ MB


**Duration outliers**

Let´s check the outliers for movies duration

In [32]:
movies.Minutes.describe()

count    221748.000000
mean         94.930047
std          34.030400
min           1.000000
25%          85.000000
50%          91.000000
75%         102.000000
max       10062.000000
Name: Minutes, dtype: float64

In [38]:
movies[movies.Minutes==10062][['originalTitle','Year','Minutes','Genres','director','averageRating','numVotes']].sort_values(by ='averageRating')

,originalTitle,Year,Minutes,Genres,director,averageRating,numVotes
134634,Qw,2019,10062,Drama,Marco Romano,8.6,21


In [37]:
movies[movies.Minutes==1][['originalTitle','Year','Minutes','Genres','director','averageRating','numVotes']].sort_values(by ='averageRating')

,originalTitle,Year,Minutes,Genres,director,averageRating,numVotes
85468,Branding Cattle,1898,1,"Documentary,Sci-Fi",James H. White,4.8,17
90780,Tönendes ABC,1932,1,No available,László Moholy-Nagy,5.8,18
211414,Tales from the Crypticon: Midnight Madness: Bl...,2015,1,"Comedy,Crime","Cole Meyer,Charlie Billadeau,Ryan J. Gilmer,Am...",6.0,6
189545,Powered By Green Smoothies,2014,1,"Adventure,Documentary,Sport",Not available,7.6,5
148974,Comedy: The Road Less Traveled,2009,1,"Documentary,Family",Michael Jr.,8.5,10


In [162]:
movies.columns

Index(['imdb_title_id', 'original_title', 'year', 'genre', 'duration',
       'all_countries', 'language', 'director', 'actors', 'description',
       'avg_vote', 'votes', 'decade', 'multicountry', 'multilang', 'country',
       'median_vote', 'votes_10', 'votes_9', 'votes_8', 'votes_7', 'votes_6',
       'votes_5', 'votes_4', 'votes_3', 'votes_2', 'votes_1'],
      dtype='object')

In [115]:
movies[movies.country=='Spain'].duration.describe()

count    2587.000000
mean       99.463858
std        13.853662
min        53.000000
25%        90.000000
50%        98.000000
75%       107.000000
max       180.000000
Name: duration, dtype: float64

**GET TOP MOVIES**

In [47]:
movies[['originalTitle','Year','Minutes','Genres','director','averageRating','numVotes']].sort_values(by =['averageRating', 'numVotes'], ascending = False).head(28)

,originalTitle,Year,Minutes,Genres,director,averageRating,numVotes
181386,Fury Blade,2004,52,Action,Not available,10.0,178
136172,Lejos de Casa pelicula Venezolana,2020,87,"Drama,History",Abner Official,10.0,126
188653,The Barn Theatre: Tomorrow's Stars Today,2017,75,"Biography,Documentary",Phil Wurtzel,10.0,109
174003,Decompression,2013,29,"Adventure,Documentary",Not available,10.0,14
138682,Mnohaya lita. Stoyko,2020,91,"Biography,Documentary,Family",Vyacheslav Bihun,10.0,13
136369,Rezk El-Bey Lik,2017,200,"Drama,History",Habib Meselemani,10.0,10
133388,Shattered Waves,2017,90,"Drama,History",Habib Mestiri,10.0,9
135696,"Fathallah TV, 10 Ans Et Une Révolution Plus Tard",2019,80,"Documentary,Drama,Music",Not available,10.0,9
135734,Bidoun 2,2014,92,Drama,Not available,10.0,9
136133,Hams Al-Maâ,2017,87,Drama,Taieb Louhichi,10.0,9


In [52]:
movies[movies.numVotes > 1000][['originalTitle','Year','Minutes','Genres','director','averageRating','numVotes']].sort_values(by =['averageRating', 'numVotes'], ascending = False).head(10)

,originalTitle,Year,Minutes,Genres,director,averageRating,numVotes
204165,The Transcendents,2018,96,"Music,Mystery,Thriller",Derek Ahonen,9.5,2002
209198,Love in Kilnerry,2019,100,Comedy,Snorri Sturluson,9.5,1550
135232,Our Scripted Life,2020,88,Comedy,David Towner,9.4,2484
130731,Jibon Theke Neya,1970,150,"Drama,Family,War",Zahir Raihan,9.4,1740
58546,The Shawshank Redemption,1994,142,Drama,Frank Darabont,9.3,2242140
91144,Hababam Sinifi,1975,87,"Comedy,Drama",Ertem Egilmez,9.3,35804
35181,The Godfather,1972,175,"Crime,Drama",Francis Ford Coppola,9.2,1546493
173312,CM101MMXI Fundamentals,2013,139,"Comedy,Documentary",Murat Dündar,9.2,43119
90530,Maya Bazaar,1957,192,"Comedy,Drama,Family",Kadiri Venkata Reddy,9.2,3680
199622,Aynabaji,2016,147,"Crime,Mystery,Thriller",Amitabh Reza Chowdhury,9.1,19447


In [53]:
movies[movies.numVotes > 50000][['originalTitle','Year','Minutes','Genres','director','averageRating','numVotes']].sort_values(by =['averageRating', 'numVotes'], ascending = False).head(10)

,originalTitle,Year,Minutes,Genres,director,averageRating,numVotes
58546,The Shawshank Redemption,1994,142,Drama,Frank Darabont,9.3,2242140
35181,The Godfather,1972,175,"Crime,Drama",Francis Ford Coppola,9.2,1546493
121946,The Dark Knight,2008,152,"Action,Crime,Drama",Christopher Nolan,9.0,2211374
37019,The Godfather: Part II,1974,202,"Crime,Drama",Francis Ford Coppola,9.0,1081305
58405,Pulp Fiction,1994,154,"Crime,Drama",Quentin Tarantino,8.9,1756398
73154,The Lord of the Rings: The Return of the King,2003,201,"Adventure,Drama,Fantasy",Peter Jackson,8.9,1584922
57082,Schindler's List,1993,195,"Biography,Drama,History",Steven Spielberg,8.9,1165600
22561,12 Angry Men,1957,96,"Crime,Drama",Sidney Lumet,8.9,655954
202882,Dag II,2016,135,"Action,Drama,War",Alper Caglar,8.9,103327
143996,Inception,2010,148,"Action,Adventure,Sci-Fi",Christopher Nolan,8.8,1965288


In [166]:
movies[movies.country=='Rest_of_the_world'][['original_title','year','director','all_countries','avg_vote', 'votes']].sort_values(by = 'avg_vote', ascending = False).head(10)

,original_title,year,director,all_countries,avg_vote,votes
64642,Love in Kilnerry,2019,"Daniel Keith, Snorri Sturluson",USA,10.0,2360
43375,Kirket,2019,Yogendra Singh,India,10.0,587
67395,Gini Helida Kathe,2019,Nagaraja Uppunda,India,9.8,425
42790,Runam,2019,Srinivas Gundareddy,India,9.7,133
43314,Fan,2019,Balavalli Darshith Bhat,India,9.6,1010
67390,Android Kunjappan Version 5.25,2019,Ratheesh Balakrishnan Poduval,India,9.6,1176
43357,The Brighton Miracle,2019,Max Mannix,Australia,9.5,617
67452,Yeh Suhaagraat Impossible,2019,Abhinav Thakur,India,9.5,635
43318,Safe,2019,Pradeep Kalipurayath,India,9.5,1017
42843,Shibu,2019,Arjun Prabhakaran,India,9.4,533


In [201]:
genres.head()

,imdb_title_id,Genre,original_title,year,duration,language,description,avg_vote,votes,decade,multicountry,multilang,country,median_vote
0,tt0000574,Biography,The Story of the Kelly Gang,1906,70,No available,True story of notorious Australian outlaw Ned ...,6.1,537,1900,0,0,Rest_of_the_world,6.0
1,tt0001892,Drama,Den sorte drøm,1911,53,No available,Two men of high rank are both wooing the beaut...,5.9,171,1910,1,0,Rest_of_the_world,6.0
2,tt0002101,Drama,Cleopatra,1912,100,English,The fabled queen of Egypt's affair with Roman ...,5.2,420,1910,0,0,Rest_of_the_world,5.0
3,tt0002130,Adventure,L'Inferno,1911,68,Italian,Loosely adapted from Dante's Divine Comedy and...,7.0,2019,1910,0,0,Rest_of_the_world,7.0
4,tt0002199,Biography,"From the Manger to the Cross; or, Jesus of Naz...",1912,60,English,"An account of the life of Jesus Christ, based ...",5.7,438,1910,0,0,Rest_of_the_world,6.0


In [214]:
genres.columns

Index(['imdb_title_id', 'Genre', 'original_title', 'year', 'duration',
       'all_countries', 'language', 'director', 'actors', 'description',
       'avg_vote', 'votes', 'decade', 'multicountry', 'multilang', 'country',
       'median_vote'],
      dtype='object')